Prometheus
----------

![](https://prometheus.io/assets/architecture.png)

Quelle: [Prometheus](https://prometheus.io) 

- - -

[Prometheus](https://prometheus.io) ist ein Open-Source-Toolkit zur Systemüberwachung und -Alamierung, das ursprünglich bei SoundCloud entwickelt wurde.

Prometheus trat der [Cloud Native Computing Foundation](https://www.cncf.io/) im Jahr 2016 als zweites gehostetes Projekt nach Kubernetes bei.

Zusammen mit zwei weiteren Produkten ist der Monitoring Open Source Stack wie folgt:

* **Prometheus** ist ein Zeitreihendatenbank- (time series database) und Überwachungstool, mit dem Metrik Endpunkte abgefragt und die von diesen Endpunkten bereitgestellten Daten abgeholt und verarbeitet werden. Daten können mit PromQL abgefragt werden.
* **Grafana** ist ein Datenvisualisierungs- und Analysetool, mit dem Sie Dashboards und Diagramme für Ihre Metrikdaten erstellen können.
* **Alertmanager**, der normalerweise zusammen mit Prometheus bereitgestellt wird, bildet die Warnschicht des Stapels. Er verarbeitet von Prometheus generierte Warnungen und dedupliziert (eliminieren von redundanten Daten), gruppiert und leitet sie z.B. via E-Mail weiter.


***
### Installation

Prometheus ist am einfachsten mittels [helm](https://helm.io) installierbar.


In [ ]:
%%bash
kubectl create namespace monitoring
helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
helm repo update
helm install monitoring prometheus-community/kube-prometheus-stack -n monitoring

Die Ports der User Interfaces

* Grafana (Service: prometheus-operator-grafana) 
* Prometheus (Service: prometheus-operator-prometheus)
* Alertmanager (Service: prometheus-operator-alertmanager)

sind nur innerhalb des Cluster zugänglich. Deshalb müssen wir diese zuerst freischalten bzw. gegen Aussen öffnen:

In [ ]:
! kubectl -n monitoring patch service monitoring-grafana                      -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-prometheus   -p '{"spec": {"type": "LoadBalancer"}}'
! kubectl -n monitoring patch service monitoring-kube-prometheus-alertmanager -p '{"spec": {"type": "LoadBalancer"}}'

Sind Sie freigeschaltet, können wir Sie wie andere Services via Cluster-IP und Port erreichen:

In [ ]:
! echo "Grafana UI     : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-grafana -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Prometheus UI  : " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')
! echo "Alertmanager UI: " http://$(cat ~/work/server-ip):$(kubectl -n monitoring get service/monitoring-kube-prometheus-alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')

**Grafana UI** Einloggen mittels User/Password **admin/prom-operator**. Auf der linken Seite kann zwischen einer Reihe von vorbereitenden Dashboards ausgewählt werden, z.B. Nodes.

In der **Prometheus** Oberfläche kann mittels der Abfragesprache [PromQL](https://prometheus.io/docs/prometheus/latest/querying/basics/) gezielt Ressourcen ausgewählt werden, z.B. durch Query von `apiserver_storage_objects`.

Der **Alertmanager** dient zum Verarbeiten von Warnungen. Für ein Beispiel siehe [Notifikation Beispiel](https://prometheus.io/docs/alerting/notification_examples/)

- - -
### Custom Ressourcen

Der Prometheus [Operator](https://github.com/coreos/prometheus-operator/blob/master/README.md) stellt eine Reihe von [benutzerdefinierten Ressourcen (CRDs)](https://kubernetes.io/docs/tasks/access-kubernetes-api/extend-api-custom-resource-definitions/) für das Monitoring bereit.

* **Prometheus**, für eine Prometheus Instanz.
* **Alertmanager**, für eine Alertmanager Instanz
* **ServiceMonitor**, für die Überwachung von Kubernetes-Diensten.
* **PodMonitor**, für die Überwachung von Pods.


In [ ]:
! kubectl api-resources | grep monitoring
! echo "---------------------------------"
! kubectl explain servicemonitors                   

- - -
### Eigene Services überwachen

Damit eigene (Micro)Services überwacht werden können, müssen diese Metrics Informationen liefern.

Dazu müssen die (Micro)Services Metrik Informationen liefern.

Die am Anfang vorgestellen synchronen Microservices liefern diese.

Dazu starten wir die Microservices frisch inkl. ServiceMonitor für Prometheus.

In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/apache.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/catalog.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/customer.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/order.yaml
kubectl apply -f https://raw.githubusercontent.com/mc-b/misegr/master/ewolff/ms-kubernetes/postgres.yaml

cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: microservice
  labels:
    tier: microservice
spec:
  selector:
    matchLabels:
      tier: microservice
  endpoints:
  - port: web
%EOF%

echo "http://$(cat ~/work/server-ip):$(kubectl get service apache --namespace ms-rest -o=jsonpath='{ .spec.ports[0].nodePort }')"

Für die nächsten Befehle müssen alle Container gestartet sein:

In [ ]:
! kubectl get all,servicemonitors --all-namespaces -l tier=microservice

Dann können wir mittels `curl` die Metriken abfragen, z.B. von Customer.

Es werden u.a. folgende Informationen ausgegeben:
* `counter.status.200.metrics` - Anzahl Aufrufe von /metrics
* `counter.status.200.list.html`- Anzahl Zugriff auf die HTML Seiten der Applikation 

In [ ]:
! curl http://customer.ms-rest:8080/metrics | grep counter

- - -
### Prometheus 

Die erstellten Microservices können wir nun mit Prometheus überwachen.

Dazu brauchen wir die entsprechenden Rechte:   

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: ServiceAccount
metadata:
  name: microservice
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: microservice
rules:
- apiGroups: [""]
  resources:
  - nodes
  - services
  - endpoints
  - pods
  verbs: ["get", "list", "watch"]
- apiGroups: [""]
  resources:
  - configmaps
  verbs: ["get"]
- nonResourceURLs: ["/metrics"]
  verbs: ["get"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: microservice
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: microservice
subjects:
- kind: ServiceAccount
  name: microservice
  namespace: ms-kubernetes
%EOF%


Und eine eigene Prometheus Instanz.

Im User Interface können wir dann die überwachten Targets und deren Status anschauen.

Dazu ist als Query `up` einzugeben oder via Pulldown -> Status -> Targets die überwachten Microservices anzuzeigen.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: v1
kind: Service
metadata:
  name: prometheus
spec:
  type: NodePort
  ports:
  - name: web
    port: 9090
    protocol: TCP
    targetPort: web
  selector:
    prometheus: prometheus
  type: LoadBalancer
---
apiVersion: monitoring.coreos.com/v1
kind: Prometheus
metadata:
  name: prometheus
spec:
  serviceAccountName: microservice
  serviceMonitorSelector:
    matchLabels:
      tier: microservice
  resources:
    requests:
      memory: 400Mi
  enableAdminAPI: true
%EOF%

echo "http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest prometheus -o=jsonpath='{ .spec.ports[0].nodePort }')"

- - -
### Prometheus Beispiel

Prometheus selber liefert ein eigenes Beispiel eines überwachten Services.

Dieser ist in diesem [Blogeintrag](https://prometheus-operator.dev/docs/user-guides/getting-started/) beschrieben.

Wir starten das Prometheus Beispiel zu Vergleichszwecken:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice   
spec:
  replicas: 3
  selector:
    matchLabels:
      app: example-app
  template:
    metadata:
      labels:
        app: example-app
        tier: microservice        
    spec:
      containers:
      - name: example-app
        image: fabxc/instrumented_app
        ports:
        - name: web
          containerPort: 8080
---
kind: Service
apiVersion: v1
metadata:
  name: example-app
  labels:
    app: example-app
    tier: microservice    
spec:
  selector:
    app: example-app
  ports:
  - name: web
    port: 8080
%EOF%

In [ ]:
! kubectl get all,servicemonitors -n ms-rest -l tier=microservice

In [ ]:
! curl example-app.ms-rest:8080/metrics